# 二维卷积层

二维互相关运算是二维卷积运算的一个特例，其中卷积核是只包含一个通道的二维矩阵。

In [1]:
from mxnet import nd,autograd
from mxnet.gluon import nn

def corr2d(X,K):
    h,w = K.shape
    Y = nd.zeros((X.shape[0]-h+1,X.shape[1]-w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+h,j:j+w]*K).sum()
    return Y

In [2]:
X = nd.array([[0,1,2],[3,4,5],[6,7,8]])
K = nd.array([[0,1],[2,3]])
print(corr2d(X,K))


[[19. 25.]
 [37. 43.]]
<NDArray 2x2 @cpu(0)>


二维卷积层

In [3]:
class Conv2D(nn.Block):
    def __init__(self,num_channels,kernel_size,**kwargs):
        super(Conv2D,self).__init__(**kwargs)
        self.weight = self.params.get('weight',shape=(num_channels,kernel_size,kernel_size))
        self.bias = self.params.get('bias',shape=(num_channels,))
    def forward(self,X):
        return corr2d(X,self.weight.data())+self.bias.data()

图像中物体的边缘检测

In [4]:
X=nd.ones((6,8))
X[:,2:6]=0
X


[[1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]]
<NDArray 6x8 @cpu(0)>

In [5]:
K=nd.array([[1,-1]]) 


[[ 1. -1.]]
<NDArray 1x2 @cpu(0)>

In [7]:
Y=corr2d(X,K)
Y
# 白到黑为正，黑到白为负，其余为0


[[ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]]
<NDArray 6x7 @cpu(0)>

通过数据学习核数组

In [13]:
conv2d=nn.Conv2D(1,kernel_size=(1,2)) # 1个通道，卷积核大小为1x2
conv2d.initialize()

X=X.reshape((1,1,6,8)) # 1个样本，1个通道，6行，8列
Y=Y.reshape((1,1,6,7))

for i in range(10):
    with autograd.record():
        Y_hat=conv2d(X)
        l=(Y_hat-Y)**2
    l.backward()
    conv2d.weight.data()[:] -= 3e-2 * conv2d.weight.grad()
    # 忽略偏差
    if (i+1)%2==0:
        print('batch %d, loss %.4f' % (i+1, l.mean().asscalar()))

batch 2, loss 0.1203
batch 4, loss 0.0202
batch 6, loss 0.0034
batch 8, loss 0.0006
batch 10, loss 0.0001


In [15]:
conv2d.weight.data().reshape((1,2))

# 初始核数组 K 是你手动定义的，而学到的核数组是通过训练数据优化得到的。
# 学到的核数组可能会与初始核数组不同，因为它是通过梯度下降法调整的，目的是最小化损失函数。
# 可以看到，本例中两者比较相近


[[ 0.9886643  -0.98795456]]
<NDArray 1x2 @cpu(0)>

互相关运算与卷积运算

* 卷积运算与互相关运算在数学上是等价的。两者的主要区别在于卷积核是否需要翻转。
* 如果没有特殊要求，两者的结果是一样的。

特征图和感受野
* 二维卷积层输出的二维数组可以看作是输入在空间维度上的一个投影。
* 二维卷积层输出的通道数与卷积核的数量相同。
* 卷积核在输入上滑动时，其参数保持不变，这称为感受野（receptive field）。